In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchinfo import summary
import torchvision
from torchvision import transforms
from prepare_data import create_dataloaders

/home/emir/miniconda3/envs/mlptorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_dir = "../datasets/"

transform_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([224])
])


train_data = torchvision.datasets.CIFAR10(root=dataset_dir,
                                         download=False,
                                         train=True,
                                         transform=transform_tensor)
test_data = torchvision.datasets.CIFAR10(root=dataset_dir,
                                        download=False,
                                        train=False,
                                        transform=transform_tensor)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
vit_l_16_model_weigths = torchvision.models.ViT_L_16_Weights.DEFAULT

vit_l_16_model = torchvision.models.vit_l_16(vit_l_16_model_weigths).to(device)

/home/emir/miniconda3/envs/mlptorch/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [5]:
summary(model=vit_l_16_model, 
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 1000]           1,024                True
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 1024, 14, 14]   787,456              True
├─Encoder (encoder)                                          [32, 197, 1024]      [32, 197, 1024]      201,728              True
│    └─Dropout (dropout)                                     [32, 197, 1024]      [32, 197, 1024]      --                   --
│    └─Sequential (layers)                                   [32, 197, 1024]      [32, 197, 1024]      --                   True
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 1024]      [32, 197, 1024]      12,596,224           True
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 1024]      [32, 197, 10

In [6]:
dataloader_transforms = vit_l_16_model_weigths.transforms

In [7]:
print(dataloader_transforms)

functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=224, resize_size=242)


In [8]:
train_dataloader, test_dataloader, class_names = create_dataloaders(train_data=train_data,
                                                                    test_data=test_data,
                                                                    transform=dataloader_transforms,
                                                                    data_folder_imported=True,
                                                                    batch_size=32)

In [9]:
image, label = next(iter(train_dataloader))

In [10]:
image.shape

torch.Size([32, 3, 224, 224])

In [11]:
for param in vit_l_16_model.parameters():
    param.requires_grad = False
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [12]:
vit_l_16_model.heads.requires_grad_ = True # unfreezing classifier

vit_l_16_model.heads = nn.Linear(in_features=1024, out_features=len(class_names), device=device)

In [13]:
summary(model=vit_l_16_model, 
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 10]             1,024                Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 1024, 14, 14]   (787,456)            False
├─Encoder (encoder)                                          [32, 197, 1024]      [32, 197, 1024]      201,728              False
│    └─Dropout (dropout)                                     [32, 197, 1024]      [32, 197, 1024]      --                   --
│    └─Sequential (layers)                                   [32, 197, 1024]      [32, 197, 1024]      --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 1024]      [32, 197, 1024]      (12,596,224)         False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 1024]      [32, 

In [14]:
def manual_seed():
    torch.cuda.manual_seed(1)
    np.random.seed(1)

In [15]:
summary(model=vit_l_16_model, 
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 10]             1,024                Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 1024, 14, 14]   (787,456)            False
├─Encoder (encoder)                                          [32, 197, 1024]      [32, 197, 1024]      201,728              False
│    └─Dropout (dropout)                                     [32, 197, 1024]      [32, 197, 1024]      --                   --
│    └─Sequential (layers)                                   [32, 197, 1024]      [32, 197, 1024]      --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 1024]      [32, 197, 1024]      (12,596,224)         False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 1024]      [32, 

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lr=0.001, params=vit_l_16_model.parameters(), momentum=0.9)
manual_seed()
from train_model import train
vit_l_16_model_results = train(model=vit_l_16_model,
                              train_dataloader=train_dataloader,
                              test_dataloader=test_dataloader,
                              optimizer=optimizer,
                              loss_fn=loss_fn,
                              device=device,
                              epochs=14)

  7%|▋         | 1/14 [13:41<2:57:58, 821.44s/it]

Epoch: 1 | train_loss: 0.3398 | train_acc: 0.9057 | test_loss: 0.2416 | test_acc: 0.9240


 14%|█▍        | 2/14 [25:17<2:29:30, 747.51s/it]

Epoch: 2 | train_loss: 0.2118 | train_acc: 0.9324 | test_loss: 0.2186 | test_acc: 0.9301


 21%|██▏       | 3/14 [36:53<2:12:46, 724.25s/it]

...

Epoch: 12 | train_loss: 0.1404 | train_acc: 0.9545 | test_loss: 0.1829 | test_acc: 0.9401


 93%|█████████▎| 13/14 [2:33:01<11:36, 696.76s/it]

Epoch: 13 | train_loss: 0.1377 | train_acc: 0.9548 | test_loss: 0.1830 | test_acc: 0.9399


100%|██████████| 14/14 [2:44:36<00:00, 705.44s/it]

Epoch: 14 | train_loss: 0.1353 | train_acc: 0.9557 | test_loss: 0.1824 | test_acc: 0.9408
